<a href="https://colab.research.google.com/github/mwojtczak/LogProxy/blob/master/Exercise_3_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%run Exercise_2_P3.ipynb

ERROR:root:File `'Exercise_2_P3.ipynb.py'` not found.


# Validation and cross-validation 

In this exercise you will implement a validation pipeline. 

At the end of Exercise 2, you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise 2.


In [0]:
#######################################################
# TODO: Implement the basic validation method,        # 
# compare MSLE on training, validation, and test sets #
#######################################################

# print(model)
# zmienna model jest zaimportowana z poprzedniego notebooka

from sklearn.model_selection import train_test_split

def test_with_districts(df, model, feature_engin=False):
    xs, ys = split_df(df, include_district=True)
    xs = one_hot_encode_district(xs)
    if feature_engin:
        xs = add_feature_engineering(xs)
    ys_predicted = model.predict(xs)

    cost_train = msle(ys, ys_predicted)
    print(f'MSLE Test cost = {cost_train} (approx. {sqrt(cost_train):.2f} ^2)')
    

def train_and_test_with_district(df, df_test, feature_engin=False):
    xs, prices = split_df(df, include_district=True)
    xs = one_hot_encode_district(xs)
    
    if feature_engin:
        xs = add_feature_engineering(xs)
        
    x_train, x_validate, y_train, y_validate = train_test_split(xs.values, prices.values, test_size=0.2)
    
    model = linear_model.SGDClassifier(loss='log', penalty=None, eta0=0.7)
    model.fit(x_train, y_train)
    ys_predicted = model.predict(x_train)
    
    cost_train = msle(y_train, ys_predicted)
    print(f'MSLE Cost = {cost_train} (approx. {sqrt(cost_train):.2f} ^2)')
    
    y_validate_predicted = model.predict(x_validate)
    cost_validate = msle(y_validate, y_validate_predicted)
    print(f'MSLE Train cost = {cost_validate} (approx. {sqrt(cost_validate):.2f} ^2)')
    
    test_with_districts(df_test, model, True)
    return model

train_and_test_with_district(df, df_test, True)



To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)


Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.


In [0]:
####################################
# TODO: Implement cross-validation # 
####################################

from sklearn.model_selection import KFold

def prepare_data(df):
    xs, ys = split_df(df, include_district=True)
    xs = one_hot_encode_district(xs)
    xs = add_feature_engineering(xs)
    return xs, ys

xs, ys = prepare_data(df)
xs_test, ys_test = prepare_data(df_test)

def get_linear_regression_model():
    return linear_model.SGDClassifier(loss='log', penalty=None, eta0=0.7)

def cross_validate_model(get_model, xs, ys, partitions=5):
    kf = KFold(n_splits=5)
    costs = []
    for train_index, test_index in kf.split(xs):
        X_train, X_test = xs.iloc[train_index], xs.iloc[test_index],
        y_train, y_test = ys[train_index], ys[test_index]
        val_cost = cross_validate_phase(get_model, X_train, y_train , X_test, y_test)
        costs.append(val_cost)
    print(f'MSLE Average cost = {sum(costs) / len(costs)} (approx. {sqrt(sum(costs) / len(costs)):.2f} ^2)')
        

def cross_validate_phase(get_model, x_train, y_train, xs_test, ys_test):
    model = get_model()
    model.fit(x_train, y_train)
    ys_predicted = model.predict(xs_test)
    cost_train = msle(ys_test.values, ys_predicted)
    print(f'MSLE Cost = {cost_train} (approx. {sqrt(cost_train):.2f} ^2)')
    
    y_validate_predicted = model.predict(xs_test)
    cost_validate = msle(ys_test.values, y_validate_predicted)
    print(f'MSLE Train cost = {cost_validate} (approx. {sqrt(cost_validate):.2f} ^2)')
    return cost_validate

cross_validate_model(get_linear_regression_model, xs, ys, 5)

    



Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [0]:
##############################
# TODO: Investigate the data #
##############################

# Jak powinnismy zdecydowac sie, ile bedzie wynosil learning rate?
# Czy należy po prostu kilka-kilkanaście razy wytrenować model dla różnych learning rate i wybrać najlepszy?
# A może napisać kod, który będzie badał, jaki lr jest najlepszy - badając z większą granulacją lry, które są bliżej lr o lepszych wynikach?
# wydaje się, że kiedy losowo inicjalizuje się wagi (a nie na 0), to może to zaburzać wyniki - niektóre lr będą miały lepszy start
# i mogą się nie okazać tak dobre dla mniej szczęśliwych losowań.